# 📄 Permeability Estimation from VTI Pore Geometry using `porePermFoam`

This notebook demonstrates the **end-to-end workflow** for running a pore-scale CFD simulation in OpenFOAM using the `porePermFoam` (via `simpleFoam_tools`) package.

We will:
1. Load a binary `.vti` pore geometry.
2. Automatically generate OpenFOAM case files.
3. Run a `simpleFoam` simulation.
4. Post-process results to compute **porosity** and **permeability**.

---

## 1️⃣ Import Required Packages
We import:
- `simpleFoam_tools` (`sft`) for case generation, meshing, simulation, and post-processing.
- `pandas` and `numpy` for handling and analyzing results.

In [1]:
import simpleFoam_tools as sft
import pandas as pd
import numpy as np

## 2️⃣ Remove Any Previous Case Files (Optional)

To start fresh, we remove any previously generated OpenFOAM case directories and results using:
`sft.remove_run_files(".")`.

This ensures no old mesh or simulation results interfere with the new run.

In [2]:
remove = 1
if remove:
    sft.remove_run_files(".")

## 3️⃣ Define Simulation Parameters

The key inputs are:

- **domain_name**: base name of the `.vti` file in `constant/geometry/`. **(Note: place your vti file in `constant/geometry/` directory)**
- **scale**: physical size of each voxel (meters).
- **dp**: differential pressure (Pa).
- **refinement**: local mesh refinement level (0 = none).
- **factor_mesh**: global mesh resolution scaling.
- **boundary_type**: `"symmetryPlane"` or `"Wall"`.
- **dt**: simulation time step (s).
- **end_time**: simulation duration (s).
- **write_interval**: how often to write simulation results (timesteps).


In [3]:
domain_name = "vti_bent_x"  # "cylinder" or "vti_bent_x"

scale = 3.0035e-6  # voxel size [m]
dp = 1.0           # pressure difference [Pa]
refinement = 0     # local refinement level
factor_mesh = 1    # global mesh scale factor

boundary_type = "symmetryPlane"  # "symmetryPlane" or "Wall"

dt = 1e-6
end_time = 500e-6
write_interval = 100

## 4️⃣ Generate OpenFOAM Case Files from `.vti` Geometry

Here we:
1. Get the voxel shape of the domain.
2. Convert `.vti` → `.stl` surface mesh.
3. Generate:
   - `blockMeshDict`
   - `snappyHexMeshDict`
   - `controlDict`
   - Initial pressure and velocity fields.

In [4]:
# Get voxel shape and a pore voxel location from VTI
vti_path = f"constant/geometry/{domain_name}.vti"
shape = sft.vti_shape(vti_path)
location_in_mesh = sft.find_pore_location(vti_path)

# Convert VTI to STL surface mesh
stl = f"{domain_name}.stl"
stl_path = f"constant/triSurface/{stl}"
sft.vti_to_stl(vti_path, stl_path)

# Adjust mesh resolution
mesh_resolution = tuple(x * factor_mesh for x in shape)

# File paths
blockMeshDict_path    = "system/blockMeshDict"
controlDict_path      = "system/controlDict"
p_field_path          = "0/p"
U_field_path          = "0/U"
snappyHexMeshDict_path = "system/snappyHexMeshDict"

# Generate OpenFOAM files
sft.generate_blockMeshDict(shape, mesh_resolution, blockMeshDict_path, boundary=boundary_type)
sft.generate_snappyHexMeshDict(location_in_mesh, stl, snappyHexMeshDict_path, refinement=refinement)
sft.generate_controlDict(controlDict_path, end_time=end_time, write_interval=write_interval, dt=dt)
sft.generate_pressure_field(p_field_path, dp=dp, boundary=boundary_type)
sft.generate_velocity_field(U_field_path, boundary=boundary_type)


Running command: pvpython /home/h09435ap/porePermFoam/simpleFoam_tools/paraview_stl.py /home/h09435ap/porePermFoam/constant/geometry/vti_bent_x.vti /home/h09435ap/porePermFoam/constant/triSurface/vti_bent_x.stl
hwloc/linux: Ignoring PCI device with non-16bit domain.
Pass --enable-32bits-pci-domain to configure to support such devices
(warning: it would break the library ABI, don't enable unless really needed).
✅ Wrote ASCII STL: /home/h09435ap/porePermFoam/constant/triSurface/vti_bent_x.stl
Generated blockMeshDict at: system/blockMeshDict
Generated snappyHexMeshDict at: system/snappyHexMeshDict
Generated controlDict at: system/controlDict
Generated p at: 0/p with boundary symmetryPlane
Generated U at: 0/U with boundary symmetryPlane


## 5️⃣ Run the Simulation

We now run the `simpleFoam` steady-state solver directly from Python using `sft.run_simplefoam`.

This step will:
- Run meshing (`blockMesh` + `snappyHexMesh`)
- Solve flow equations.
- Output results in CSV for post-processing.

In [5]:
sft.run_simplefoam(".", scale=scale)


>>> blockMesh

/*---------------------------------------------------------------------------*\
| =========                 |                                                 |
| \\      /  F ield         | OpenFOAM: The Open Source CFD Toolbox           |
|  \\    /   O peration     | Version:  2412                                  |
|   \\  /    A nd           | Website:  www.openfoam.com                      |
|    \\/     M anipulation  |                                                 |
\*---------------------------------------------------------------------------*/
Build  : _f00230df-20250528 OPENFOAM=2412 patch=250528 version=2412
Arch   : "LSB;label=32;scalar=64"
Exec   : blockMesh
Date   : Aug 08 2025
Time   : 13:38:31
Host   : e-10aux32876g5
PID    : 2181884
I/O    : uncollated
Case   : /home/h09435ap/porePermFoam
nProcs : 1
trapFpe: Floating point exception trapping enabled (FOAM_SIGFPE).
fileModificationChecking : Monitoring run-time modified files using timeStampMaster (file

## 6️⃣ Load Simulation Results

The solver outputs a `q_in.csv` file containing:
- Cross-sectional area.
- Volumetric flow rate.

We read it with `pandas` for further calculations.

In [6]:
df_q_area = pd.read_csv("q_in.csv")
df_q_area.head()

,time,flowRate_phi,area
0,0.0000,-4.005890e-12,1.657780e-08
1,0.0001,-4.121360e-12,1.657780e-08
2,0.0002,-4.127120e-12,1.657780e-08
3,0.0003,-4.127420e-12,1.657780e-08
4,0.0004,-4.127440e-12,1.657780e-08


## 7️⃣ Compute Porosity and Permeability

**Porosity** is computed directly from the voxel geometry.  
**Permeability** is computed from Darcy’s law:


$$k = \frac{Q \, \mu \, L}{A \, \Delta P}$$


Where:
- $Q$ = volumetric flow rate
- $\mu$ = fluid viscosity (Pa·s)
- $L$ = sample length (m)
- $A$ = cross-sectional area (m²)
- $\Delta P$ = pressure difference (Pa)

In [ ]:
phi = sft.vti_phi(vti_path)
print(f"Porosity: {phi:.02%}")

# Cross-sectional area
A = shape[1] * shape[2] * scale**2
miu = 1e-3  # fluid viscosity [Pa·s]
L = shape[0] * scale  # length [m]

# Flow rate from last timestep
Q = np.abs(df_q_area["flowRate_phi"].iloc[-1])

# Darcy's law
k = Q * miu * L / (A * dp)

print(f"Permeability: {k:.02e} m^2")
print(f"Permeability: {k * 1.01324997e15:.02f} md")

Porosity: 24.81%
Permeability: 1.37e-11 m^2
Permeability: 13924.18 md
